**DEB**

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from Data_challenge_fairness_2024.evaluator import *
from evaluator import *

In [2]:
#####################################################
# Load pickle file and convert to numpy array
#####################################################

with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)

#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

X_test_true = dat['X_test']
S_test_true = dat['S_test']

Y56= Y + 28*S
#X, X_test,Y,S, S_test = dat[1]

print(X.shape,Y.shape,S.shape,X_test_true.shape,S_test_true.shape)

path_model = 'models/'

dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


**MODELS**
---
<br>
0_Baseline : logistic regression (on X_train)<br>
1_Baseline optimised : optimized logistic regression (on X + hyperparameters)<br>
2_Orthogonal : optimised logisitic regression (on orthogonal X)<br>

In [3]:
path_model = 'FINAL_models/'  # must finish by '/'
path_Y_pred_true = 'FINAL_Y_pred_true/'  # must finish by '/'

# with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)
# with open(path_model + name + '.pkl', 'rb') as f: model = pickle.load(f)

def get_scores(Y_pred, Y, S):
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
    macro_f1 = eval_scores['macro_fscore']
    inv_macro_gap = 1-eval_scores['TPR_GAP']

    #print results
    print('final score :',final_score)
    print('macro_f1    :',eval_scores['macro_fscore'])
    print('inv_macro_gap',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, macro_f1, inv_macro_gap

**LOGISTIC REGRESSION**
---

In [4]:
###############################################
# 0. LOGISTIC REGRESSION - BASELINE 
###############################################

name ='0_Reglog_baseline' # changer clf_i

# Refresh training data
X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=42)

# training (or load)logistic model
#clf_0 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train, Y_train)
with open(path_model + name + '.pkl', 'rb') as f: clf_0 = pickle.load(f)
model = clf_0 
  
# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
Y_pred_true = model.predict(X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

Accuracy on transformed test data: 0.7638438438438439
final score : 0.7365033273594812
macro_f1    : 0.669337208333607
inv_macro_gap 0.8036694463853555


In [5]:
##################################################
# 1. LOGISTIC REGRESSION - BASELINE OPTIMISED
# (no validation set - training on all X)
##################################################

name ='1_Reglog_optimised' # changer clf_i

# Refresh training data
# TRAINED ON ALL X

# training (or load)logistic model
# added regularisation 'l2' with coeff 0.2 (C) and early stopping with tol = 0.0001
clf_1 = LogisticRegression(random_state=42, max_iter=5000,verbose=1,penalty='l2', C=0.2, tol=0.0001).fit(X, Y)
# with open(path_model + name + '.pkl', 'rb') as f: clf_1 = pickle.load(f)
model = clf_1

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
Y_pred_true = model.predict(X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.24653D+04    |proj g|=  4.11128D+04


 This problem is unconstrained.



At iterate   50    f=  2.05318D+04    |proj g|=  8.07961D+02

At iterate  100    f=  1.73620D+04    |proj g|=  6.71832D+02

At iterate  150    f=  1.61758D+04    |proj g|=  1.03107D+02

At iterate  200    f=  1.57705D+04    |proj g|=  1.24845D+02

At iterate  250    f=  1.56370D+04    |proj g|=  3.30421D+01

At iterate  300    f=  1.55878D+04    |proj g|=  2.25705D+01

At iterate  350    f=  1.55716D+04    |proj g|=  3.01638D+01

At iterate  400    f=  1.55661D+04    |proj g|=  1.65593D+01

At iterate  450    f=  1.55641D+04    |proj g|=  1.12923D+01

At iterate  500    f=  1.55632D+04    |proj g|=  3.90256D+00

At iterate  550    f=  1.55628D+04    |proj g|=  6.05409D+00

At iterate  600    f=  1.55624D+04    |proj g|=  4.51413D+00

At iterate  650    f=  1.55619D+04    |proj g|=  1.26648D+01

At iterate  700    f=  1.55610D+04    |proj g|=  7.07335D+00

At iterate  750    f=  1.55595D+04    |proj g|=  1.62580D+01

At iterate  800    f=  1.55577D+04    |proj g|=  4.04886D+00

At iter

In [8]:
##########################################################################
# 3. MODEL 2 + ORTHOGONAL PROJECTION 
# (using the vector of the average difference of embedding between groups 
# to debiase representation of X, by operating an orthogonal projection
##########################################################################

name ='2_LogReg_optimised_orthogonal' # changer clf_i

# CREATION OF ORTHOGONAL PROJECTION 
# ---------------------------------
# Comparison of average embedding

#calculate average embedding for group sensitive (S=1) and non sensitive (S=0)
X_sensitive = X[S==1]
X_sensitive_mean=X_sensitive.mean()

X_non_sensitive = X[S!=1]
X_non_sensitive_mean=X_non_sensitive.mean()

diff_mean = X_sensitive_mean-X_non_sensitive_mean

# function to debiase data
def remove_info(dataset, diff):
    # Compute the dot product of each row of the dataset with diff
    dot_products = np.dot(dataset, diff)
    
    # Compute the magnitude of diff
    diff_magnitude_squared = np.dot(diff, diff)
    
    # Compute the projection of each row onto diff
    projection = np.outer(dot_products / diff_magnitude_squared, diff)
    
    # Subtract the projection from the dataset
    modified_dataset = dataset - projection
    
    return modified_dataset

# Debiasing X (Projection of X orthogonally to diff) 
# projection in new representation space
modified_X = remove_info(X, diff_mean)

# TRAINING MODEL AND PREDICTING
#------------------------------------------------------------------------

clf_2 = LogisticRegression(random_state=42, max_iter=5000,verbose=1,penalty='l2', C=0.2, tol=0.0001).fit(modified_X, Y)
# with open(path_model + name + '.pkl', 'rb') as f: clf_2 = pickle.load(f)
model = clf_2

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
modified_X_test_true = remove_info(X_test_true, diff_mean)  # debiasing
Y_pred_true = model.predict(modified_X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.24653D+04    |proj g|=  4.11537D+04


 This problem is unconstrained.



At iterate   50    f=  2.09922D+04    |proj g|=  1.42201D+03

At iterate  100    f=  1.75913D+04    |proj g|=  4.34926D+02

At iterate  150    f=  1.64928D+04    |proj g|=  1.25458D+02

At iterate  200    f=  1.60922D+04    |proj g|=  1.01404D+02


In [ ]:
###############################################
# ORTHOGONAL PROJECTION - MAN-WOMEN
###############################################
# 
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import tokenizers as tk
from tqdm import tqdm

# Charger le tokenizer 
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Charger le modèle
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

# Tokenize "man" and "woman"
tokens_man = tokenizer.encode("man", add_special_tokens=True)  # Ajoute les tokens spéciaux
tokens_woman = tokenizer.encode("woman", add_special_tokens=True)

print(tokens_man)
print(tokens_woman)

# transforme en tensor de batch (meme si un seul mot)

input_ids_man = tf.constant(tokens_man)[None, :]  # Ajoute une dimension de batch
input_ids_woman = tf.constant(tokens_woman)[None, :]  # Ajoute une dimension de batch

# Obtenir les embeddings
outputs_man = model(input_ids_man)
outputs_woman = model(input_ids_woman)

# Les embeddings du dernier layer pour le premier token ('[CLS]' par défaut)
embedding_man = outputs_man.last_hidden_state[0][0]
embedding_woman = outputs_woman.last_hidden_state[0][0]

print(embedding_man.shape)  # Doit être (768,)
print(embedding_woman.shape)  # Doit être (768,)


In [ ]:
#print(X_sensitive_mean[:10])
#print(X_debiaised_sensitive_mean[:10])
#print(X_non_sensitive_mean[:10])

# Create DataFrame
compare = pd.DataFrame({'X_sensitive': X_sensitive_mean, 'Difference S/non S': diff_mean, 'X_debiaised': X_debiaised_sensitive_mean,'X_non_sensitive':X_non_sensitive_mean})
compare

import numpy as np

In [ ]:
#####################################################
#   SCORE A BLANC
#####################################################


n=Y.shape[0]
Y_pred =np.ones(n)*1
accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy}")
eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

#print results
print('final score',final_score)
print('macro_fscore',eval_scores['macro_fscore'])
print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])

In [ ]:
# test score "prediction uniforme"

n=Y.shape[0]

Scores_U=pd.DataFrame(columns=['N','N_f','N_h','accuracy','final_score','macro_f1','macro_gap'])

print(Scores_U)
for i in range(28):
    #Test value for all i values
    test=pd.DataFrame(np.ones(11893,dtype=int)*i)
    test.to_csv("all/Data_Challenge_all_"+str(i)+".csv", header = None, index = None)
    
    Y_pred=pd.DataFrame(np.ones(n,dtype=int)*i)
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    macro_f1 = eval_scores['macro_fscore']
    macro_gap = eval_scores['TPR_GAP']
    final_score = (macro_f1 +1- macro_gap)/2
    # check number of occurence
    N = (Y==i).sum()
    N_f = (Y56 == i + 28).sum()
    N_h = (Y56 == i).sum()
    Scores_U.loc[i]= [N, N_f, N_h, accuracy,final_score,macro_f1,macro_gap]

Scores_U

In [ ]:
size = Scores_U['N']
score = Scores_U['final_score']


# Créer et entraîner le modèle de régression linéaire
from sklearn.linear_model import LinearRegression
reverse = LinearRegression()
reverse.fit(np.array(score).reshape(-1, 1),np.array(size).reshape(-1, 1))

# Load the scores for X_true
scores_true = pd. read_csv('all/distribution_Y_true.txt',header = None,index_col=0)
dist_true_pred = reverse.predict(scores_true) #/ 11893 * 27749

# Afficher la distribution de X_test_true
plt.scatter(size,score)
plt.scatter(dist_true_pred,scores_true)
plt.show

In [ ]:
plt.scatter(Scores_U['final_score'],scores_true)

In [ ]:
plt.plot(range(28),size,label='X')
plt.plot(range(28),np.round(dist_true_pred),'X_test_true')
plt.legend()

In [ ]:
# Normalize data (L2 norm recommended for embeddings)
#X = normalize(X, norm='l2')
#X_test_true = normalize(X_test_true, norm='l2')



In [ ]:
##############################################################
# train_test_split with Y56 (np.arrays)
##############################################################

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y56_train, Y56_test = train_test_split(X, Y56, test_size=0.2, random_state=42)
Y_train = Y56_train % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train = Y56_train//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_test = Y56_test % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_test = Y56_test//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

# impression des dimensions
print('train:',X_train.shape,Y_train.shape,S_train.shape)
print('test:',X_test.shape,Y_test.shape, S_test.shape)

**Explore data**
---

In [ ]:
dist = pd.DataFrame({'label':Y,'S':S})

# Sum counts of 1s and 0s to get the total count for each label
result = dist.groupby('label')['S'].value_counts().unstack(fill_value=0)
result.columns = ['S', 'not_S']
result['total_count'] = result.sum(axis=1)

# Calculate totals for each column
totals = result.sum(axis=0)
result.loc[30] = totals

# Calculate total count percentages for each count
result['%_S_label'] = round((result['S'] / result['total_count']) * 100)
result['%_not_S_label'] = round((result['not_S'] / result['total_count']) * 100)
result['%_total'] = np.round(result['total_count']/len(Y)*100,2 ) # % of total count
result['%_S_total'] = np.round(result['S']/(S==1).sum()*100,2 ) # % of total count
result['%_not_S_total'] = np.round(result['not_S']/(S!=1).sum()*100,2 ) # % of total count
result['diff_%_S_total']=result['%_S_total']-result['%_total']
result['|']='|'
#Reorder table
result = result.reindex(columns=['total_count', '%_total','|','S','%_S_total','diff_%_S_total','|','not_S', '%_not_S_total','|','%_S_label', '%_not_S_label'])


result

In [ ]:
# Extract the data from the '%_S' column
labels = pd.to_numeric(result.index)
data = result['%_total']
data_S = result['%_S_total']
data_not_S = result['%_not_S_total']

# Create a bar plot
plt.plot(labels[:-1], data[:-1])
plt.plot(labels[:-1],data_S[:-1], label='Sensitive')
plt.plot(labels[:-1],data_not_S[:-1], label='Not_sensitive')

# Add labels and title
plt.xlabel('Label')
plt.ylabel('Percentage of total')
plt.title('Percentage of total for sample, S and non-S')
plt.legend()
# Show the plot
plt.show()

In [ ]:
result_sorted = result.iloc[:-1].sort_values(by='diff_%_S_total', ascending=False)
result_sorted['original_label'] = result_sorted.index
result_sorted=result_sorted.reset_index(drop=True)
#result_sorted.reset_index(drop=True, inplace=True)
result_sorted

In [ ]:
# Extract the data from the '%_S' column
labels_sorted = result_sorted['original_label']
#print(labels_sorted)
data = result_sorted['%_total']
data_S = result_sorted['%_S_total']
data_not_S = result_sorted['%_not_S_total']

# Create a bar plot
plt.plot(data)
plt.plot(data_S, label='Sensitive')
plt.plot(data_not_S, label='Not_sensitive')

# Add labels and title
plt.xlabel('Label')
plt.ylabel('Percentage of total')
#plt.xticks(labels_sorted)
plt.xticks(ticks=range(len(labels_sorted)), labels=labels_sorted, rotation=90)  # Rotate if there are many labels

plt.title('Percentage of total for sample, S and non-S')
plt.legend()
# Show the plot
plt.show()

**MODEL ORTHOGONAL DEBIASING**
---

In [ ]:
import torch


# Fonction de similarité cosinus
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Pré-générer les embeddings pour chaque token dans le vocabulaire
# ATTENTION : Cette étape est très coûteuse !
vocab_embeddings = {}
for word, token_id in tokenizer.vocab.items():
    input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(word)]).unsqueeze(0)  # Ajouter batch dimension
    with torch.no_grad():
        outputs = model(input_ids)
    vocab_embeddings[word] = outputs.last_hidden_state[0, 0, :].numpy()  # Utiliser l'embedding du token

# Supposons que X soit votre matrice d'embeddings (n, 768)
# X = ...

# Trouver le mot le plus proche pour les 10 premiers embeddings
for i in range(10):
    embedding = X[i]
    similarities = {word: cosine_similarity(embedding, word_emb) for word, word_emb in vocab_embeddings.items()}
    closest_word = max(similarities, key=similarities.get)
    print(f"Ligne {i}: Mot le plus proche = {closest_word}")


**baseline Logistic regression on raw data**
---

In [ ]:
print("Coefficients:", clf_1.coef_.shape)
print("Intercept:", clf_1.intercept_.shape)

In [ ]:
print(eval_scores_1)
print(confusion_matrices_eval_1.keys)

#show confusion matrix for Accurarcy key 0
pd.DataFrame(confusion_matrices_eval_1[0])

In [ ]:
# (Re)Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

# Classify the provided test data with you classifier
y_test_true_1 = clf_1.predict(X_test_true)
results_1=pd.DataFrame(y_test_true_1, columns= ['score'])

results_1.to_csv("Data_Challenge_MDI_341_1.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

**SECOND METHOD - ADVERSARIAL NN**
---

In [ ]:
# Refresh training data
# X_train, X_test, Y_train, Y_test, S_train, S_test = X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy , CategoricalCrossentropy

from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical

# Assuming X_train is your input embeddings and S is your sensitive attribute

# Input layer
input_layer = Input(shape=(768,))

# Main task classifier layers
main_task_hidden = Dense(256, activation='relu')(input_layer)
main_task_output = Dropout(0.5)(main_task_hidden)
main_task_output = Dense(28, activation='softmax', name='main_task_output')(main_task_hidden)

# Adversarial component layers
adversary_hidden = Dense(256, activation='relu')(main_task_hidden)
adversary_hidden = Dropout(0.5)(adversary_hidden)
adversarial_output = Dense(1, activation='sigmoid', name='adversarial_output')(adversary_hidden)

# Model
model_2 = Model(inputs=input_layer, outputs=[main_task_output, adversarial_output])

# Optimizers
#main_task_optimizer = Adam(learning_rate=0.001)
#adversarial_optimizer = Adam(learning_rate=0.001)

# Loss functions
main_task_loss = CategoricalCrossentropy()
adversarial_loss = BinaryCrossentropy()

# Compile the model
model_2.compile(optimizer=Adam(learning_rate=0.001),
              loss={'main_task_output': main_task_loss, 'adversarial_output': adversarial_loss},
              loss_weights=[1, -0.1],
              metrics={'main_task_output': ['accuracy'], 'adversarial_output': [AUC()]})

# Prepare the labels for the main task and the adversarial task
Y_main_task = to_categorical(Y_train, num_classes=28)#Y_train # Your main task labels
Y_adversary = S_train    # Your sensitive attribute labels

# check size on input .output
print(X_train.shape,Y_main_task.shape,Y_adversary.shape)

# X normal:isation
# none in method 2

# Train the model
model_2.fit(X_train, {'main_task_output': Y_main_task, 'adversarial_output': Y_adversary}, epochs=10)

# After training, you can use the output of `main_task_hidden` as your new unbiased representation


In [ ]:
# Step 1: Transform X_train and X_test
feature_extractor_2 = Model(inputs=model_2.input, outputs=main_task_hidden)
X_train_transformed_2 = feature_extractor_2.predict(X_train)
X_test_transformed_2 = feature_extractor_2.predict(X_test)
X_test_true_transformed_2 = feature_extractor_2.predict(X_test_true)

# Step 2: Train a new classifier on the transformed training data
clf_2 = LogisticRegression(max_iter=5000)  # Increase max_iter if needed for convergence
history_new_2 = clf_2.fit(X_train_transformed_2, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_2 = clf_2.predict(X_test_transformed_2)
accuracy_2= accuracy_score(Y_test, Y_pred_2)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_2}")

# Step 4 : Predict with gloabl score
eval_scores_2, confusion_matrices_eval_2 = gap_eval_scores(Y_pred_2, Y_test, S_test, metrics=['TPR'])
final_score_2 = (eval_scores_2['macro_fscore']+ (1-eval_scores_2['TPR_GAP']))/2
print('\nfinal',final_score_2)
print('macro_fscore',eval_scores_2['macro_fscore'])
print('1-eval_scores',1-eval_scores_2['TPR_GAP'])

In [ ]:
# Classify the provided test data with you classifier
y_test = clf_2.predict(X_test_true_transformed_2)
results=pd.DataFrame(y_test, columns= ['score'])

results.to_csv("Data_Challenge_MDI_341_2.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

**THIRD METHOD**
---

In [ ]:
from sklearn.preprocessing import StandardScaler

# get X_train an
# X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=42)

# Initialize StandardScaler
scaler = StandardScaler()

# Fit the scaler to your training data
scaler.fit(X_train)

# Transform your training data
X_train_standardized = scaler.transform(X_train)
X_test_standardized = scaler.transform(X_test)
X_test_true_standardized = scaler.transform(X_test_true)



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy , CategoricalCrossentropy

from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical

# Assuming X_train is your input embeddings and S is your sensitive attribute

# Input layer
input_layer = Input(shape=(768,))

# Main task classifier layers
main_task_hidden = Dense(256, activation='relu')(input_layer)
main_task_hidden = Dropout(0.5)(main_task_hidden)
main_task_output = Dense(28, activation='softmax', name='main_task_output')(main_task_hidden)

# Adversarial component layers
adversary_hidden = Dense(256, activation='relu')(main_task_hidden)
adversary_hidden = Dropout(0.5)(adversary_hidden)
adversarial_output = Dense(1, activation='sigmoid', name='adversarial_output')(adversary_hidden)

# Model
model_3 = Model(inputs=input_layer, outputs=[main_task_output, adversarial_output])

# Optimizers
#main_task_optimizer = Adam(learning_rate=0.001)
#adversarial_optimizer = Adam(learning_rate=0.001)

# Loss functions
main_task_loss = CategoricalCrossentropy()
adversarial_loss = BinaryCrossentropy()

# Compile the model
model_3.compile(optimizer=Adam(learning_rate=0.001),
              loss={'main_task_output': main_task_loss, 'adversarial_output': adversarial_loss},
              metrics={'main_task_output': ['accuracy'], 'adversarial_output': [AUC()]})

# Prepare the labels for the main task and the adversarial task
Y_main_task = to_categorical(Y_train, num_classes=28)#Y_train # Your main task labels
Y_adversary = S_train    # Your sensitive attribute labels

# check size on input .output
print(X_train_standardized.shape,Y_main_task.shape,Y_adversary.shape)

# Train the model
model_3.fit(X_train_standardized, {'main_task_output': Y_main_task, 'adversarial_output': Y_adversary}, epochs=10)

# After training, you can use the output of `main_task_hidden` as your new unbiased representation


In [ ]:
# Step 1: Transform X_train and X_test
feature_extractor_3 = Model(inputs=model_3.input, outputs=main_task_hidden)
X_train_transformed_3 = feature_extractor_3.predict(X_train_standardized)
X_test_transformed_3 = feature_extractor_3.predict(X_test_standardized)
X_test_true_transformed_3 = feature_extractor_3.predict(X_test_true_standardized)

# Step 2: Train a new classifier on the transformed training data
new_classifier_3 = LogisticRegression(max_iter=10000)  # Increase max_iter if needed for convergence
history_new_3 = new_classifier_3.fit(X_train_transformed_3, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_3 = new_classifier_3.predict(X_test_transformed_3)
accuracy_3= accuracy_score(Y_test, Y_pred_3)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_3}")

In [ ]:
# calculate final score
eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred_3, Y_test, S_test, metrics=['TPR'])
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

print('macro_fscore',eval_scores['macro_fscore'])
print('1-eval_scores',1-eval_scores['TPR_GAP'])
print('final score (average)',final_score)

In [ ]:
# Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

X_test_true_transformed_3 = feature_extractor_3.predict(X_test_true)

# Classify the provided test data with you classifier
y_test_true = clf.predict(X_test_true_transformed_3)
results_3=pd.DataFrame(y_test_true, columns= ['score'])

results_3.to_csv("Data_Challenge_MDI_341_3.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

**4th METHOD**
---

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy , CategoricalCrossentropy

from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical

# Assuming X_train is your input embeddings and S is your sensitive attribute

# Input layer
input_layer = Input(shape=(768,))

# Main task classifier layers
main_task_hidden = Dense(512, activation='relu')(input_layer)
main_task_hidden = Dropout(0.5)(main_task_hidden)
main_task_output = Dense(28, activation='softmax', name='main_task_output')(main_task_hidden)

# Adversarial component layers
adversary_hidden = Dense(512, activation='relu')(main_task_hidden)
adversary_hidden = Dropout(0.5)(adversary_hidden)
adversarial_output = Dense(1, activation='sigmoid', name='adversarial_output')(adversary_hidden)

# Model
model_4 = Model(inputs=input_layer, outputs=[main_task_output, adversarial_output])

# Optimizers
#main_task_optimizer = Adam(learning_rate=0.001)
#adversarial_optimizer = Adam(learning_rate=0.001)

# Loss functions
main_task_loss = CategoricalCrossentropy()
adversarial_loss = BinaryCrossentropy()

# Compile the model
model_4.compile(optimizer=Adam(learning_rate=0.001),
              loss={'main_task_output': main_task_loss, 'adversarial_output': adversarial_loss},
              metrics={'main_task_output': ['accuracy'], 'adversarial_output': [AUC()]})

# Prepare the labels for the main task and the adversarial task
Y_main_task = to_categorical(Y_train, num_classes=28)#Y_train # Your main task labels
Y_adversary = S_train    # Your sensitive attribute labels

# check size on input .output
print(X_train_standardized.shape,Y_main_task.shape,Y_adversary.shape)

# Train the model
model_4.fit(X_train_standardized, {'main_task_output': Y_main_task, 'adversarial_output': Y_adversary}, epochs=10)

# After training, you can use the output of `main_task_hidden` as your new unbiased representation


In [ ]:
# Step 1: Transform X_train and X_test
feature_extractor_4 = Model(inputs=model_4.input, outputs=main_task_hidden)
X_train_transformed_4 = feature_extractor_4.predict(X_train_standardized)
X_test_transformed_4 = feature_extractor_4.predict(X_test_standardized)
X_test_true_transformed_4 = feature_extractor_4.predict(X_test_true_standardized)

# Step 2: Train a new classifier on the transformed training data
new_classifier_4 = LogisticRegression(max_iter=10000)  # Increase max_iter if needed for convergence
history_4 = new_classifier_4.fit(X_train_transformed_4, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_4 = new_classifier_4.predict(X_test_transformed_4)
accuracy_4 = accuracy_score(Y_test, Y_pred_4)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_4}")

In [ ]:
eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred_4, Y_test, S_test, metrics=['TPR'])
#eval_scores#eval_scores['macro_fscore']
#eval_scores['TPR_GAP']
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
print(final_score)

In [ ]:
# Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

X_test_true_transformed_4 = feature_extractor_4.predict(X_test_true)

# Classify the provided test data with you classifier
y_test_true = clf.predict(X_test_true_transformed_4)
results_4=pd.DataFrame(y_test_true, columns= ['score'])

results_4.to_csv("Data_Challenge_MDI_341_4.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')